In [1]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras import optimizers
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential, Model, load_model
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import warnings
warnings.filterwarnings('ignore')
from dog_images import DogImages
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
picsize = 224
# Step 1: Get List of Dogs
lst = [x[0] for x in os.walk('../Images')]
lst_dogs = [a.replace('../Images/', '') for a in lst[1:]]

dog_images = DogImages(lst_dogs, picsize)
# dog_images.generate_img_files()
train_imgs = dog_images.load_images('train')
test_imgs = dog_images.load_images('test')
Xtest = test_imgs[0]
Ytest = test_imgs[1]
Xtrain = train_imgs[0]
Ytrain = train_imgs[1]

lst = [x[0] for x in os.walk('../Images')]
lst_dogs = [a[a.index('-')+1:] for a in lst[1:]]
print(lst_dogs[0])


Tibetan_terrier


In [3]:
Xtrain_pro = np.zeros(Ytrain.shape)
Xtest_pro = np.zeros(Ytest.shape)

Xtrain_pre = np.zeros(Ytrain.shape)
Xtest_pre = np.zeros(Ytest.shape)

Xtrain.shape

(31662, 150528)

In [4]:
def shuffle_function(X, y):
    Xhold = X.copy()
    Yhold = y.copy()
    new = np.array([i for i in range(Xhold.shape[0])])
    np.random.shuffle(new)
    for i, n in enumerate(new):
        X[i, :] = Xhold[n, :]
        y[i, :] = Yhold[n, :]
    return X, y

In [5]:
Xtest, Ytest = shuffle_function(Xtest, Ytest)
Xtrain, Ytrain = shuffle_function(Xtrain, Ytrain)

In [6]:
import random

def my_generator(X, y, batch_size):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 224, 224, 3))
    batch_labels = np.zeros((batch_size,y.shape[1]))
    features = X.shape[0]
    indices = []
    while True:
        for i in range(batch_size):
            # choose random index in features
            try:
                index = indices[0]
                indices = indices[1:]
            except:
                indices = [a for a in range(X.shape[0] - 1)]
                random.shuffle(indices)
                index = indices[0]
                indices = indices[1:]
            batch_features[i] = X[index,:].reshape(1,224,224,3)
            batch_labels[i] = y[index,:]
        yield batch_features, batch_labels

In [7]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD, Adam

# create the base pre-trained model
# base_model = InceptionV3(weights='imagenet', include_top=True)#
base_model = VGG16(weights='imagenet', include_top=True)

# add a global spatial average pooling 
base_model.layers.pop()
# x = base_model.output
x = base_model.layers[-1].output
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1000, activation='relu')(x)
# and a logistic layer -- let's say we have 120 classes
predictions = Dense(120, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
for layer in model.layers[:len(model.layers[:])-1]:
    layer.trainable = False
model.summary()

'''
This whole mess was due to include_top=False
# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#     layer.trainable = False

# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')"

# # train the model on the new data for a few epochs
# Y_train = np.argmax(Ytrain)
# model.fit_generator(my_generator(Xtrain, Ytrain, 200), samples_per_epoch=50, nb_epoch=10)

# # at this point, the top layers are well trained and we can start fine-tuning
# # convolutional layers from inception V3. We will freeze the bottom N layers
# # and train the remaining top layers.

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True
'''
# for layer in model.layers[:len(model.layers[:])-1]:
#     layer.trainable = False
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(), loss='categorical_crossentropy')
bs = 32
samp = int(Xtrain.shape[0]/bs)
print(samp)
model.fit_generator(my_generator(Xtrain, Ytrain, 32), samples_per_epoch=1000, nb_epoch=10, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
try:
    preds = model.predict(X_test, verbose=1)
    print('ding!')
except:
    X_test = np.zeros((Xtest.shape[0],224,224,3))
    for i in range(Xtest.shape[0]):
        X_test[i,:,:,:] = Xtest[i,:].reshape(1,224,224,3)
        print('\r {:.0f}%'.format(100*(i+1)/Xtest.shape[0]), end="")
    preds = model.predict(X_test, verbose=1)
    print('ding!')

9498/9498 [==============================] - 890s   
ding!


In [9]:
model.evaluate

<bound method Model.evaluate of <keras.engine.training.Model object at 0x7f5e9dd9eb70>>

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [10]:
np.sum((np.argmax(preds,1) == np.argmax(Ytest,1)))/Ytest.shape[0]

0.47083596546641399

In [11]:
np.argmax(preds,1)

array([  5,   7,  89, ...,  86,  16, 101])

In [12]:
np.argmax(Ytest,1)

array([99,  7, 89, ..., 32, 65, 98])

In [13]:
model.evaluate(X_test, Ytest)

9498/9498 [==============================] - 1007s  


2.26670131625113